In [1]:

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import geopandas as gpd
import os


df = pd.read_csv('/kaggle/input/world-happiness-report-2023/WHR2023.csv')
# countries = np.array(report['Country name'])
df
# print(len(countries)) 



/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


,Country name,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,7.804,0.036,7.875,7.733,10.792,0.969,71.150,0.961,-0.019,0.182,1.778,1.888,1.585,0.535,0.772,0.126,0.535,2.363
1,Denmark,7.586,0.041,7.667,7.506,10.962,0.954,71.250,0.934,0.134,0.196,1.778,1.949,1.548,0.537,0.734,0.208,0.525,2.084
2,Iceland,7.530,0.049,7.625,7.434,10.896,0.983,72.050,0.936,0.211,0.668,1.778,1.926,1.620,0.559,0.738,0.250,0.187,2.250
3,Israel,7.473,0.032,7.535,7.411,10.639,0.943,72.697,0.809,-0.023,0.708,1.778,1.833,1.521,0.577,0.569,0.124,0.158,2.691
4,Netherlands,7.403,0.029,7.460,7.346,10.942,0.930,71.550,0.887,0.213,0.379,1.778,1.942,1.488,0.545,0.672,0.251,0.394,2.110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,Congo (Kinshasa),3.207,0.095,3.394,3.020,7.007,0.652,55.375,0.664,0.086,0.834,1.778,0.531,0.784,0.105,0.375,0.183,0.068,1.162
133,Zimbabwe,3.204,0.061,3.323,3.084,7.641,0.690,54.050,0.654,-0.046,0.766,1.778,0.758,0.881,0.069,0.363,0.112,0.117,0.905
134,Sierra Leone,3.138,0.082,3.299,2.976,7.394,0.555,54.900,0.660,0.105,0.858,1.778,0.670,0.540,0.092,0.371,0.193,0.051,1.221
135,Lebanon,2.392,0.044,2.479,2.305,9.478,0.530,66.149,0.474,-0.141,0.891,1.778,1.417,0.476,0.398,0.123,0.061,0.027,-0.110


Now let's visualize that data 

# Check for he missing data
for col in df:
    percentage_missing = np.mean(df[col].isnull())
    print('{} missing --> {}'.format(col,percentage_missing)

In [ ]:
countries = [country for country in report['Country name']]
limit = 20
sns.relplot(y='Country name',x='Ladder score',data=df[:20],palette='Set2')
sns.relplot(y='Country name',x='Ladder score',data=df[-20:])

First thing that stands out is that most of the countries with **high ladder** score are **european**, whereas the ones with low score are in majority from **Africa**

It isn't surprising considering e.g. GDP of these countries

In [ ]:
print('GDP of TOP 20 -->',df['Logged GDP per capita'][:20].mean())
print('GDP of BOTTOM 20 -->',df['Logged GDP per capita'][-20:].mean())
print('Mean of all the countries -->', df['Logged GDP per capita'].mean())

In [ ]:
ladder_to_GDP = df[['Ladder score','Logged GDP per capita']]

sns.regplot(y='Ladder score',x='Logged GDP per capita',data=df)
print('Now lets check the greater anomalies on this chart')


Now we can see some anomalous values. Let us detect it



In [ ]:
x=df[['Logged GDP per capita']]
y=df['Ladder score']


model = LinearRegression().fit(x, y)
residuals = y - model.predict(x)
outliers = df[np.abs(residuals) > 2 * residuals.std()].index

df[['is_outlier']] = 0
df.loc[outliers,'is_outlier'] = 1
print(df[df['is_outlier']==1][['Country name','Ladder score','Logged GDP per capita']])

outliers = df[df['is_outlier'] == 1]
sns.relplot(y='Ladder score',x='Logged GDP per capita',data=df,hue='is_outlier')

# Label the outliers
for index, row in outliers.iterrows():
    plt.text(row['Logged GDP per capita'], row['Ladder score'], row['Country name'])

print('\n Correlation of Ladder score to GDP per capita --> ',df.corr()['Ladder score']['Logged GDP per capita'])
df_no_outliers = df[df['is_outlier'] == 0]
print('\n Correlation of Ladder score to GDP per capita after dropping rows with outliers --> ',df_no_outliers.corr()['Ladder score']['Logged GDP per capita'])

Take a look at correlation heatmap

In [ ]:
correlation = df.drop(labels=['is_outlier','Standard error of ladder score','upperwhisker','lowerwhisker'],axis=1).corr()


print(correlation['Ladder score'].sort_values(ascending=False))
fig,ax = plt.subplots(figsize=(9,8))
correlation = correlation[(correlation<0.999)]
sns.heatmap(correlation,cmap='RdBu',annot=True,vmin=-1,vmax=1,center=0)

That was a heatmap with outliers, now let's take a look at the one that omit the outliers

In [ ]:
correlation = df_no_outliers.drop(labels=['is_outlier','Standard error of ladder score','upperwhisker','lowerwhisker'],axis=1).corr()
fig,ax = plt.subplots(figsize=(9,8))
correlation = correlation[(correlation>-2) & (correlation<0.999)]
print(correlation['Ladder score'].sort_values(ascending=False))
sns.heatmap(correlation,cmap='RdBu',annot=True,vmin=-1,vmax=1,center=0)